In [1]:
import numpy as np

In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers, models

# Load annotations
annotations_path = 'path/to/your/annotations.csv'
annotations = pd.read_csv(annotations_path, delimiter=';')

# Load and preprocess images
data = []
labels = []

for index, row in annotations.iterrows():
    image_path = 'path/to/your/images/' + row['Filename']
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, (32, 32))  # Resize image to a common size

    data.append(image.numpy())
    labels.append(row['ClassId'])

data = np.array(data)
labels = np.array(labels)

# Normalize pixel values to be between 0 and 1
data = data / 255.0

# One-hot encode labels
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

# Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the CNN model using function with parameters
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Get input shape and number of classes
input_shape = (32, 32, 3)  # Adjust based on your image size and channels
num_classes = len(np.unique(labels))

# Build the model
model = build_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f'Test accuracy: {test_acc}')

# Now, you can use the trained model for predictions
# For example, if you have a new image 'new_image_path'
new_image_path = 'path/to/your/new_image.ppm'
new_image = tf.io.read_file(new_image_path)
new_image = tf.image.decode_image(new_image, channels=3)
new_image = tf.image.resize(new_image, (32, 32))
new_image = np.expand_dims(new_image.numpy(), axis=0)  # Add batch dimension

# Preprocess the new image
new_image = new_image / 255.0

# Make prediction
prediction = model.predict(new_image)
predicted_class = np.argmax(prediction)

print(f'Predicted class: {predicted_class}')


array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [2]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np




In [8]:
loaded_model = tf.keras.models.load_model(r'C:\Users\BHG81HC\Documents\Private_Hy\Graduate_Thesis\Documents\my_model')
loaded_model2 = tf.keras.models.load_model(r'C:\Users\BHG81HC\Documents\Private_Hy\Graduate_Thesis\Documents\Model_CNN\my_model_cnn.h5')


In [22]:
# print(loaded_model2.GetSummaryInformation)
loaded_model2.summary()
loaded_model2.compile()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         204928

In [ ]:
# Đoán trên dữ liệu test
y_pred = loaded_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Chuyển đổi one-hot encoded labels thành dạng số nguyên
y_test_classes = np.argmax(y_test, axis=1)

# Tạo ma trận nhầm lẫn (Confusion Matrix)
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

# Hiển thị ma trận nhầm lẫn bằng seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Bảng thống kê
class_report = classification_report(y_test_classes, y_pred_classes, target_names=class_labels)
print("Classification Report:\n", class_report)

In [7]:
# Biểu đồ độ chính xác (Accuracy) theo epoch trong quá trình huấn luyện
plt.plot(loaded_model.history.history['accuracy'])
plt.plot(loaded_model.history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Biểu đồ hàm mất mát (Loss) theo epoch
plt.plot(loaded_model.history.history['loss'])
plt.plot(loaded_model.history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

AttributeError: 'NoneType' object has no attribute 'history'

In [24]:
import cv2

# Mở camera máy tính
cap = cv2.VideoCapture(0)  # 0 là ID của camera tích hợp trong máy tính

while True:
    # Đọc khung hình từ camera
    ret, frame = cap.read()

    # Hiển thị khung hình
    cv2.imshow('Camera Feed', frame)

    # Thoát vòng lặp khi nhấn phím 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng camera và đóng cửa sổ hiển thị
cap.release()
cv2.destroyAllWindows()